# Ping Pong Delay


In [5]:
# Setup
from scipy.io import wavfile
import numpy as np
import sounddevice as sd
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
import sys
import requests
import io
from CircularBuffer import CircularBuffer

# URL of the raw wave file on GitHub
# un-comment the URL of the file you want to use
# ATF Drums
#github_url = "https://raw.githubusercontent.com/kevbrnen/Music-Technology-Final-Project/main/Sound%20Examples/Testing_Files/Around%20the%20fur%20drums.wav"
# KYW inst
github_url = "https://raw.githubusercontent.com/kevbrnen/Music-Technology-Final-Project/main/Sound%20Examples/Testing_Files/KYW%20Inst.wav"

# Fetch the wave file from GitHub
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    # Read the wave data from the response content
    wave_data = io.BytesIO(response.content)
    
    # Read the wave file 
    fs, data = wavfile.read(wave_data)
    
else:
    print("Failed to fetch the wave file from GitHub")


# Get the bit depth of the audio data (assumes audio_data is a NumPy array)
bit_depth = data.dtype.itemsize * 8

# Calculate the scaling factor for normalization
scaling_factor = 2 ** (bit_depth - 1)  # For signed audio

# Convert audio data to floating-point values and normalize
data = data.astype(np.float32) / scaling_factor

# Verify that the data is now in the range -1 to 1
print(f"Min Value: {np.min(data)}")
print(f"Max Value: {np.max(data)}")

Min Value: -0.9484677314758301
Max Value: 0.9998999834060669


In [2]:
sd.play(data, fs)

In [6]:
# Setting up delay lines and parameters
wet_amt = 0.7
delay_ms = 100

delaySamps = ((fs/1000) * delay_ms)

Lbuffer = CircularBuffer(delaySamps)
Rbuffer = CircularBuffer(delaySamps)

fdbk_amt = 0.8

In [7]:
# Processing Loop
output = np.zeros((len(data), 2))

for i in range(len(output)):
    delayedL = Lbuffer.get_sample_from_buffer()
    delayedR = Rbuffer.get_sample_from_buffer()

    #Pushing the newest right channel sample to the left buffer and vice versa
    Lbuffer.push_sample_to_buffer((delayedR * fdbk_amt) + data[i, 1])
    Rbuffer.push_sample_to_buffer((delayedL * fdbk_amt) + data[i, 0])

    output[i,0] = ((1-wet_amt) * data[i,0]) + ((wet_amt) * delayedL)
    output[i,1] = ((1-wet_amt) * data[i,1]) + ((wet_amt) * delayedR)

#Normalisation
max_amplitude = np.max(np.abs(output))
normalized_audio = output / max_amplitude

In [23]:
sd.play(normalized_audio, fs)